In [1]:
from rdflib import Graph, Namespace
import graphene
from pyvis.network import Network


# Load the RDF graph
g = Graph()
try:
    # Replace 'iprok.owl' with the actual path to your RDF file
    g.parse("filled_bimonto.owl", format="xml")   # Adjust format if needed (e.g., 'turtle', 'n3')
    print("RDF data loaded successfully.")
except Exception as e:
    print(f"Error loading RDF file: {e}")
    exit()
    
# Function to extract the local part of a URI
def extract_local_name(uri):
    return uri.split('/')[-1].split('#')[-1]

# Graphene GraphQL Schema
class RDFNode(graphene.ObjectType):
    subject = graphene.String()
    predicate = graphene.String()
    obj = graphene.String()

# Graphene GraphQL Query (Dynamic)
class Query(graphene.ObjectType):
    all_triples = graphene.List(
        RDFNode,
        predicate_filter=graphene.String(),
        subject_filter=graphene.String(),
        object_filter=graphene.String()
    )
    
    def resolve_all_triples(self, info, predicate_filter=None, subject_filter=None, object_filter=None):
        triples = []
        for subj, pred, obj in g:
            local_subj = extract_local_name(str(subj))
            local_pred = extract_local_name(str(pred))
            local_obj = extract_local_name(str(obj))
            # Apply AND filtering logic
            if predicate_filter and local_pred != predicate_filter:
                continue
            if subject_filter and local_subj != subject_filter:
                continue
            triples.append(
                RDFNode(
                    subject=local_subj,
                    predicate=local_pred,
                    obj=local_obj
                )
            )
        return triples

schema = graphene.Schema(query=Query)


# Example GraphQL Query with Filter
query = '''
{
  allTriples{
    subject
    predicate
    obj
  }
}
'''

# Execute the query
result = schema.execute(query)


RDF data loaded successfully.


In [2]:
# Define the predicates, subjects, and data types to filter out
predicates_to_filter = {
    "type",  # rdf:type
}

subjects_to_filter = {
    "Thing",  # Exclude 'Thing' nodes
}

data_types_to_filter = {
    "boolean",
    "string",
    "integer",
    "decimal"
}

# Create a PyVis Network object with better layout settings
net = Network(height='800px', width='100%',)

# Define colors for subjects and objects
subject_color = "#FF5733"  # Orange-red
object_color = "#33FF57"   # Green

# Add nodes and edges, filtering out specified predicates, subjects, and data types
for triple in result.data['allTriples']:
    subject = triple['subject']
    predicate = triple['predicate']
    obj = triple['obj']
    
    # Skip triples with specified predicates, subjects, or data types
    if predicate in predicates_to_filter or obj in data_types_to_filter or subject in subjects_to_filter:
        continue
    
    # Add nodes with specific colors (skip 'Thing' class nodes)
    if subject not in subjects_to_filter:
        net.add_node(subject, label=subject, size=20, color=subject_color)
    if obj not in subjects_to_filter:
        net.add_node(obj, label=obj, size=20, color=object_color)
    
    # Add edges with predicate as title and label (only if neither subject nor object is filtered)
    if subject not in subjects_to_filter and obj not in subjects_to_filter:
        net.add_edge(subject, obj, title=predicate)

# Configure physics settings to improve layout
net.force_atlas_2based(gravity=0.1, central_gravity=0.1)
net.repulsion(node_distance=100)


# Show the interactive graph in the notebook
net.show("rdf_graph_filled.html")
